In [445]:
import pandas as pd
import numpy as np 
import altair as alt
import warnings 
import seaborn as sns
from scipy import stats
import requests
import os
import pdfplumber
warnings.filterwarnings('ignore')
from urllib.request import urlopen
import zipfile
import glob
import re
import requests
import xlrd
import openpyxl
from openpyxl import load_workbook

#pd.set_option('display.max_columns', 100)
pd.options.display.float_format = '{:.2f}'.format
pd.set_option("display.max_columns", 330)

# County Level data 

### Load in 990 data 

In [446]:
data_990 = pd.read_csv("processed_data/processed_990_filtered.csv") #from Clean_990_data - updated to be those iwth income over 25k and assets over 0 
data_990.FIPS = data_990.FIPS.astype(str).str.zfill(5)

In [447]:
## add county level ACS 
county_ACS = pd.read_csv("raw_data/county_level_ACS.csv",dtype={"Geo_FIPS":str})#from social explorer, B01003,B03002 from ACS, SE_B13004 from social explorer data for pov rate
county_ACS["County Name"] = county_ACS["Geo_NAME"] + " " + county_ACS["Geo_STUSAB"].str.upper()
county_ACS.Geo_FIPS = county_ACS.Geo_FIPS.astype(str).str.zfill(5)

county_ACS = county_ACS[["County Name","Geo_FIPS",
 'SE_B13004_001',
 'SE_B13004_002',
 "ACS19_5yr_B01003001",
 "ACS19_5yr_B25003001",
  "ACS19_5yr_B25003002",
   "ACS19_5yr_B25003003",
    "ACS19_5yr_B03002003",
           "ACS19_5yr_B03002004",
           "ACS19_5yr_B03002005",
            "ACS19_5yr_B03002006",
           "ACS19_5yr_B03002007",
           "ACS19_5yr_B03002009",
           "ACS19_5yr_B03002013",
           "ACS19_5yr_B03002015",
           "ACS19_5yr_B03002019"]]

county_ACS.rename(columns={"SE_B13004_001":"Total_pop_poverty_universe_2019",
                           "SE_B13004_002":"Total_pop_below_poverty_line_2019",
                           "ACS19_5yr_B01003001":"Total_pop_2019",
                           "ACS19_5yr_B25003001":"Total_Households_2019",
                           "ACS19_5yr_B25003002":"Total_Owner_Occupied_2019",
                           "ACS19_5yr_B25003003":"Total_Renter_Occupied_2019",
                           "ACS19_5yr_B03002003":"Total_NH_White_2019",
                            "ACS19_5yr_B03002004":"Total_NH_Black_2019",
                           "ACS19_5yr_B03002006":"Total_NH_Asian_2019",
                            "ACS19_5yr_B03002005": "Total_NH_American_Indian_2019",
                           "ACS19_5yr_B03002007":"Total_NH_Pacific_Islander_2019",
                           "ACS19_5yr_B03002009":"Total_NH_Two_or_More_Races_2019",
                          "ACS19_5yr_B03002013":"Total_Hispanic_White_2019",
                           "ACS19_5yr_B03002015":"Total_Hispanic_Black_2019",
                           "ACS19_5yr_B03002019":"Total_Hispanic_Two_or_More_Races_2019",
                           "Geo_FIPS":"FIPS"
                        },inplace=True)
county_ACS.replace({"County Name":{"LaSalle Parish LA":"La Salle Parish LA"}},inplace=True)
data_990.replace({"County Name":{"Shannon County SD":"Oglala Lakota County SD",
                                "DoÐa Ana County NM":"Doña Ana County NM",
                                'Wade Hampton Census Area AK':'Kusilvak Census Area AK'}},inplace=True)
county_ACS["State"]  = county_ACS["County Name"].str[-2:]
county_ACS = county_ACS[county_ACS.State != "PR"]

#merge data 
data_990_final = county_ACS.merge(data_990,on="FIPS",how="left")
data_990_final.rename(columns={'Total Population':"Total_pop_2020"},inplace=True)

def post_merge_col_name_fix(df):
    """
    Inputs:
        df (dataframe) - Dataframe post merge where the same named cols with _x and _y are identical in values

    This function looks at the typical naming conventions that occur during a merge with the same column names and fixes these inputs. 
    """
    for col in df.columns:
        df.rename(columns={col:col.replace("_x","")},inplace=True)
    
    df.drop(columns=[x for x in df.columns if "_y" in x],inplace=True)

    return df 
data_990_final = post_merge_col_name_fix(data_990_final)  
addme = pd.read_csv("raw_data/county_augment_rent.csv")# additional ACS data from Social Explorer (ACS B19013)
addme.Geo_STUSAB = addme.Geo_STUSAB.str.upper()
addme["County Name"] = addme["Geo_NAME"]+" " + addme.Geo_STUSAB
addme["Geo_FIPS"] = addme["Geo_FIPS"].astype(str).str.zfill(5)
addme = addme[["Geo_FIPS","ACS19_5yr_B19013001","ACS19_5yr_B25064001"]]
addme.rename(columns={"ACS19_5yr_B19013001":"cnty_med_income",
                     "ACS19_5yr_B25064001":"cnty_med_rent",
                     "Geo_FIPS":"FIPS"},inplace=True)
#addme.replace({"County Name":{"LaSalle Parish LA":"La Salle Parish LA"}},inplace=True)
data_990_final = data_990_final.merge(addme,on=["FIPS"],how="left")

#data_990_final["Percent_Renter_Occupied"] = data_990_final["Total_Renter_Occupied_2019"] / data_990_final["Total_Households_2019"]
#data_990_final["Percent_Owner_Occupied"] = data_990_final["Total_Owner_Occupied_2019"] / data_990_final["Total_Households_2019"]
#data_990_final["Percent_NH_White"] = data_990_final["Total_NH_White_2019"]/data_990_final["Total_pop_2019"]
#data_990_final["Percent_NH_Black"] = data_990_final["Total_NH_Black_2019"]/data_990_final["Total_pop_2019"]
#data_990_final["Percent_NH_Asian"] = data_990_final["Total_NH_Asian_2019"]/data_990_final["Total_pop_2019"]
##data_990_final["Percent_Hispanic_White"] = data_990_final["Total_Hispanic_White_2019"]/data_990_final["Total_pop_2019"]
#data_990_final["Percent_Below_Poverty_Line"] = data_990_final["Total_pop_below_poverty_line_2019"]/data_990_final["Total_pop_poverty_universe_2019"]

### Load in PUMS and HUD data to get percent of people below 80% of poverty line by county

In [448]:
#url = 'https://api.census.gov/data/2019/acs/acs5/pums?get=GRNTP,HINCP,NP,WGTP,PUMA,ST'
#response = requests.get(url)
#data_1 = pd.DataFrame(response.json())
#data_1.to_csv("all_PUMA_PUMS_AMI_check.csv")

In [449]:
df_2019 = pd.read_csv("raw_data/all_PUMA_PUMS_AMI_check.csv") #this loads in results of API call before 

In [450]:
def make_header(df):
    """
    do some basic cleanup of the header row 
    """
    new_header = df.iloc[0] #grab the first row for the header
    df = df[1:] #take the data less the header row
    df.columns = new_header
    
    df.rename(columns={"GRNTP":"gross_rent",
                     "ST":"state_code",
                     "HINCP":"hh_income",
                     "NP":"ppl_in_hh",
                     "WGTP":"weight"}, inplace=True)


    return df
df_2019 = make_header(df_2019)

In [451]:
def import_clean_crosswalk(location,county_col,puma_col):
    """
    inputs:
        location (str): Location of crosswalk csv
        county_col (str): name of column with counties 
        puma_col (str): name of column with PUMAs
    Reads in and cleans up crosswalk files from geocorr, returning subset of columns
    """
    #PUMA - County Crosswalk URL from https://mcdc.missouri.edu/applications/geocorr2018.html with puma12 & county14
    crosswalk_county = pd.read_csv(location)
    crosswalk_county = crosswalk_county.iloc[1:]
    crosswalk_county["cntyname"] = crosswalk_county["cntyname"].apply(lambda x: x.rsplit(' ',1)[0])
    #crosswalk_county[county_col] = crosswalk_county[county_col].str.replace(r' CA', '')
    crosswalk_county = crosswalk_county[[puma_col,county_col,"afact","state"]]
    crosswalk_county.rename(columns={puma_col:"PUMA",county_col:"County"},inplace=True)
    return crosswalk_county

#create PUMA county crosswalk for the seperate years 
crosswalk_county_2019 = import_clean_crosswalk("raw_data/geocorr2018_2129103863.csv",'cntyname',"puma12")

In [452]:
def crosswalk_weighting(df,crosswalk):
    """
    inputs:
        df (dataframe): PUMS data 
        crosswalk (dataframe): crosswalk of PUMAs to Counties with weights 
        puma_col (str): name of column with PUMAs
    Splits dataset between places with 1 to 1 PUMA matches to counties, does those matches, and those 
    with many to 1 matches with weights, multiplies by the weights by the weights, and then concats the dataset
    """
    
    #for crosswalk and df need to create combo of PUMA and state code, as PUMA only unique in state
    crosswalk.PUMA = crosswalk.PUMA.astype(str).str.zfill(5)
    crosswalk.state = crosswalk.state.astype(str).str.zfill(2)
    crosswalk["PUMA_state"] = crosswalk.state + crosswalk.PUMA
    
    crosswalk.afact = crosswalk.afact.astype(float)

    df.PUMA = df.PUMA.astype(str).str.zfill(5)
    df.state_code = df.state_code.astype(str).str.zfill(2)
    df["PUMA_state"] = df.state_code + df.PUMA
    
    df.weight = df.weight.astype(int)
    
    
    #clean merger
    clean_crosswalks = crosswalk[crosswalk.afact.astype(float)==1]
    clean_crosswalks_PUMAs = clean_crosswalks.PUMA_state.unique()
    df_clean_merger = df[df.PUMA_state.isin(clean_crosswalks_PUMAs)]
    df_clean = pd.merge(df_clean_merger, clean_crosswalks, on='PUMA_state', how='left')
    
    #weighted crosswalks 
    weighted_crosswalks = crosswalk[crosswalk.afact <1]
    weighted_crosswalks = weighted_crosswalks[weighted_crosswalks.afact != 0]
    weighted_crosswalks_PUMA = weighted_crosswalks.PUMA_state.unique()
    df_weighted = df[df.PUMA_state.isin(weighted_crosswalks_PUMA)]

    df_weighted = df_weighted.merge(weighted_crosswalks, on='PUMA_state',how="left")
    df_weighted["weight"] = df_weighted["weight"] * df_weighted["afact"]
    df_weighted = df_weighted[df_weighted.weight != 0]

    
    full_data = pd.concat([df_clean,df_weighted])
    full_data['county_name'] = full_data.County + " County"
    return full_data

df_2019 = crosswalk_weighting(df_2019,crosswalk_county_2019)

In [453]:
#read in state/state code mappings 
state_codes = pd.read_csv("https://gist.githubusercontent.com/dantonnoriega/bf1acd2290e15b91e6710b6fd3be0a53/raw/11d15233327c8080c9646c7e1f23052659db251d/us-state-ansi-fips.csv")
state_codes[" st"] = state_codes[" st"].astype(str).str.zfill(2)
state_codes.drop(columns=["stname"],inplace=True)
state_codes.rename(columns={" st":"state_code",
                           " stusps":"stab"},inplace=True)
df_2019.drop(columns=["PUMA_y","state"],errors="ignore",inplace=True)

#merge on state codes
df_2019 = df_2019.merge(state_codes,on="state_code",how="left")

In [454]:
df_2019.gross_rent = df_2019.gross_rent.astype(int)
df_2019["Renter"] = np.where(df_2019["gross_rent"]>0,1,0)
df_2019_income = df_2019[["stab","county_name","hh_income","ppl_in_hh","gross_rent","weight","Renter"]]
df_2019_income["county_state"] = df_2019_income["county_name"] + df_2019_income["stab"]
df_2019_income["ppl_in_hh"]  = df_2019_income["ppl_in_hh"].astype(str)
#load in HUD income limits 
HUD_INCOME_LIMITS_2019 = pd.read_excel("raw_data/Section8-FY19.xlsx")#from https://www.huduser.gov/portal/datasets/il.html
HUD_INCOME_LIMITS_2019["FIPS"] = HUD_INCOME_LIMITS_2019["State"].astype(str).str.zfill(2) + HUD_INCOME_LIMITS_2019["County"].astype(str).str.zfill(3)
HUD_INCOME_LIMITS_2019.rename(columns={"State_Alpha":"stab","County_Name":"county_name"},inplace=True)
HUD_INCOME_LIMITS_2019["county_state"] = HUD_INCOME_LIMITS_2019["county_name"] + " " + HUD_INCOME_LIMITS_2019["stab"]
HUD_INCOME_LIMITS_2019.replace({"county_state":{"La Salle County IL":"LaSalle County IL",
                                                          "Petersburg Census Area AK":"Petersburg Borough AK",
                                                           "Dona Ana County NM":"Doña Ana County NM",
                                                           "St. Mary Parish LA":"St. Mary Parish LA"}},inplace=True)
HUD_INCOME_LIMITS_2019_sub = HUD_INCOME_LIMITS_2019[['county_state','l50_1', 'l50_2', 'l50_3', 'l50_4',
       'l50_5', 'l50_6', 'l50_7', 'l50_8']]

In [455]:
#need to create income limits for greater than 8 
#For each person over eight-persons, the four-person income limit
#should be multiplied by an additional 8 percent. (For example, the nine-person limit equals 140
#percent [132 + 8] of the relevant four-person income limit.) HUD rounds income limits up to the
#nearest $50.
#https://www.huduser.gov/portal/datasets/il/il21/IncomeLimitsMethodology-FY21.pdf
big_fam = {"l50_9":1.4,
           "l50_10":1.48,
           "l50_11":1.56,
           "l50_12":1.64,
           "l50_13":1.72,
           "l50_14":1.8,
           "l50_15":1.88,
           "l50_16":1.96,
           "l50_17":2.04,
           "l50_18":2.12,
           "l50_19":2.2,
           "l50_20":2.28}

def round_nearest_50(x, base=50):
    return base * round(x/base)

for i in big_fam:
    HUD_INCOME_LIMITS_2019_sub[i] = round_nearest_50(HUD_INCOME_LIMITS_2019_sub["l50_4"] * big_fam[i])
    
HUD_INCOME_LIMITS_2019_sub_melt = HUD_INCOME_LIMITS_2019_sub.melt(id_vars=['county_state'])
HUD_INCOME_LIMITS_2019_sub_melt['ppl_in_hh'] = HUD_INCOME_LIMITS_2019_sub_melt.variable.apply(lambda x: x.split('_')[1])

In [456]:
#manually export list of mismatched county names to match up 
#matching = list(set(df_2019_income.county_state.unique()).difference(set(HUD_INCOME_LIMITS_2019_sub_melt.county_state.unique())))
#matching2= list(set(HUD_INCOME_LIMITS_2019_sub_melt.county_state.unique()).difference(set(df_2019_income.county_state.unique())))
#match1 = pd.DataFrame({ 'df_2019_income': matching })
#match2 = pd.DataFrame({'HUD_INCOME_LIMITS_2019': matching2})
#match_me = pd.concat([match1,match2],axis=1)
#match_me.to_csv("match_me.csv")

#output of manually matching mismapped PUMS -> HUD county Names 
county_mapping = pd.read_csv("raw_data/county_name_mapping_PUMS_to_HUD.csv")
county_mapping = dict(zip(county_mapping.df_2019_income,county_mapping.new_hud))
df_2019_income.county_state = df_2019_income.county_state.map(county_mapping).fillna(df_2019_income.county_state)
#fix DC 
df_2019_income.replace({"county_state":{"District of Columbia County DC":"District of Columbia DC"}},inplace=True)

In [457]:
#merge HUD income limits and PUMS data; find percentage of households below 80%AMI
HUD_INCOME_LIMITS_2019_sub_melt.ppl_in_hh = HUD_INCOME_LIMITS_2019_sub_melt.ppl_in_hh.astype(int)
df_2019_income.ppl_in_hh = df_2019_income.ppl_in_hh.astype(int)
df_2019_income.hh_income = df_2019_income.hh_income.astype(int)

merged_hud = df_2019_income.merge(HUD_INCOME_LIMITS_2019_sub_melt,on=["county_state","ppl_in_hh"],how="left")
merged_hud["Sub_50%_AMI"] = np.where(merged_hud["hh_income"]<merged_hud["value"],1,0)

merged_hud_weighted_avg = merged_hud.groupby(["county_state"]).apply(lambda x: sum(x["Sub_50%_AMI"]*x.weight)/sum(x.weight)).reset_index()
merged_hud_weighted_avg.rename(columns={0:"Percent_Sub_50%_AMI","county_state":"County Name"},inplace=True)
merged_hud_weighted_avg["Percent_Sub_50%_AMI"] = merged_hud_weighted_avg["Percent_Sub_50%_AMI"] * 100

merged_hud_renter = merged_hud[merged_hud.Renter == 1]
merged_hud_weighted_avg_renter = merged_hud_renter.groupby(["county_state"]).apply(lambda x: sum(x["Sub_50%_AMI"]*x.weight)/sum(x.weight)).reset_index()
merged_hud_weighted_avg_renter.rename(columns={0:"Percent_Sub_50%_AMI_Renter","county_state":"County Name"},inplace=True)
merged_hud_weighted_avg_renter["Percent_Sub_50%_AMI_Renter"] = merged_hud_weighted_avg_renter["Percent_Sub_50%_AMI_Renter"] * 100

In [458]:
#these are county state pairs in the HUD data not in the 990 data;
#set(merged_hud_weighted_avg["County Name"]).difference(set(data_990["County Name"]))

In [459]:
#merged_hud_weighted_avg.to_csv("merged_hud_weighted_avg.csv")
#merged_hud_weighted_avg_renter.to_csv("merged_hud_weighted_avg_renter.csv")

In [460]:
merged_hud_weighted_avg.replace({"County Name":{"La Salle County IL":"LaSalle County IL",
                                               "Petersburg Census Area AK":"Petersburg Borough AK",
                                               "Dona Ana County NM":"Doña Ana County NM"}},inplace=True)

In [461]:
#merge onto 990 data
data_990_final = data_990_final.merge(merged_hud_weighted_avg,on="County Name",how="left")
data_990_final = data_990_final.merge(merged_hud_weighted_avg_renter,on="County Name",how="left")

### Add BLS Labor Force Data 

Data below labor force data by county, not seasonally adjusted, August 2019–September 2020
From: https://www.bls.gov/lau/#data

In [462]:
## get unemployment data 

# read in data
old_bls = pd.read_csv("raw_data/BLS_2019_2020.csv", dtype={'state': str,"county":str})
old_bls = old_bls[old_bls.Period == "Feb-20"]
old_bls.rename(columns={"state":"State","county":"County"},inplace=True)

bls_df = pd.read_csv("raw_data/bls_latest.csv", 
                     dtype={'State':str, 'County': str})
bls_df = bls_df[bls_df.Period.isin(['Feb-21','Oct-21', 'Nov-21 p'])]


old_bls= old_bls.iloc[:-3]
bls_df = bls_df.iloc[:-5]


bls_df = pd.concat([old_bls,bls_df])

# clean columns
bls_df["FIPS"] = bls_df["State"] + bls_df["County"]
bls_df['month'] = bls_df.Period.str.split("-", expand=True)[0]
bls_df['year'] = '20'+bls_df.Period.str.split("-", expand=True)[1].str[:2]
bls_df.rename(columns={'Labor Force':'c_labor',
                       'Employed':'c_employed',
                       'Unemployed':'c_unemployed',
                       'Unemployment Rate (%)':'p_unemployed'}, inplace=True)

# convert unemployment % to float
bls_df['p_unemployed'] = bls_df.p_unemployed.str.strip().replace('–','0').astype(float)

unemployment_fields = []
for i in bls_df.Period.unique():
    temp = bls_df.loc[bls_df.Period==i].copy()
    
    month = temp.Period.str.split("-",expand=True).iloc[0][0]
    #print(month)
    year = temp.Period.str.split("-",expand=True).iloc[0][1][:2]
    #print(year)
    #unemployment_fields.append('p_unemployed_'+month+year)
    temp.rename(columns={'p_unemployed':'p_unemployed_'+month+year,
                        "c_labor":'c_labor_'+month+year,
                        'c_employed':'c_employed_'+month+year,
                        'c_unemployed':'c_unemployed_'+month+year},inplace=True)
    
    #print(temp.columns)

    data_990_final = data_990_final.merge(temp[['FIPS', 'p_unemployed_'+month+year,"c_labor"+"_"+month+year,"c_employed"+"_"+month+year,"c_unemployed"+"_"+month+year]],
                                            on=['FIPS'],
                                            how='left')
    
    data_990_final = data_990_final.replace(',','', regex=True)

In [463]:
def post_merge_col_name_fix(df):
    """
    Inputs:
        df (dataframe) - Dataframe post merge where the same named cols with _x and _y are identical in values

    This function looks at the typical naming conventions that occur during a merge with the same column names and fixes these inputs. 
    """
    for col in df.columns:
        df.rename(columns={col:col.replace("_x","")},inplace=True)
    
    df.drop(columns=[x for x in df.columns if "_y" in x],inplace=True)

    return df 

data_990_final = post_merge_col_name_fix(data_990_final)

### Load in CDBG county level 

In [464]:
cdbd_county_2019 = pd.read_csv("raw_data/county_cdbg_2019.csv")
cdbd_county_2020 = pd.read_csv("raw_data/county_cdbg_2020.csv")
cdbd_county = cdbd_county_2019.merge(cdbd_county_2020,on=["NAME","STA"])
cdbd_county = post_merge_col_name_fix(cdbd_county) 
cdbd_county["County Name"] = cdbd_county["NAME"] + " " + cdbd_county["STA"]

In [465]:
#these are the 10 not porting over correctly 
set(cdbd_county["County Name"]).difference(set(data_990_final["County Name"]))

{'Athens-Clarke County GA',
 'Augusta-Richmond County GA',
 'De Kalb County GA',
 'Du Page County IL',
 'Honolulu HI',
 'Mchenry County IL',
 'Prince Georges County MD',
 'St Clair County IL',
 'St Louis County MN',
 'St Louis County MO'}

In [466]:
#manually fix them 
cdbd_county.replace({"County Name":{'Athens-Clarke County GA':"Clarke County GA",
                                    'Augusta-Richmond County GA':"Richmond County GA",
                                   'De Kalb County GA':"DeKalb County GA",
                                   'Du Page County IL':"DuPage County IL",
                                   'Honolulu HI':"Honolulu County HI",
                                   'Mchenry County IL':'McHenry County IL',
                                   'Prince Georges County MD':"Prince George's County MD",
                                   'St Clair County IL':'St. Clair County IL',
                                   'St Louis County MN':'St. Louis County MN',
                                   'St Louis County MO':'St. Louis County MO'}},inplace=True)

In [467]:
cdbd_county =cdbd_county[['CDBG19', 'HOME19',
       'ESG19', 'HOPWA19', 'HTF19', 'CDBG20',
       'RHP20', 'HOME20', 'ESG20', 'HOPWA20', 'HTF20', 'CDBG_CV1', 'CDBG_CV2',
       'CDBG_CV3', 'ESG_CV1', 'ESG_CV2', 'HOPWA_CV', 'County Name']]

In [468]:
data_990_final = data_990_final.merge(cdbd_county,on="County Name",how="left")

In [469]:
data_990_final = data_990_final[[
 'County Name',
 'Number of Nonprofits',
 'Total Assets of Nonprofits',
 'Total Income of Nonprofits in Area',
 'FIPS',
 'Assets_per_capita',
 'Income_per_capita',
 'Nonprofits_per_1k',
# 'Percent_Sub_50%_AMI',
# 'Percent_Sub_50%_AMI_Renter',
"cnty_med_income","cnty_med_rent",
    'p_unemployed_Feb20', 
       'p_unemployed_Feb21', 
      'p_unemployed_Oct21',
       'p_unemployed_Nov21', 
    "Percent_Sub_50%_AMI",
"Percent_Sub_50%_AMI_Renter",
 'Total_pop_poverty_universe_2019',
 'Total_pop_below_poverty_line_2019',
 'Total_pop_2019',
 'Total_Households_2019',
 'Total_Owner_Occupied_2019',
 'Total_Renter_Occupied_2019',
 'Total_NH_White_2019',
 'Total_NH_Black_2019',
 'Total_NH_American_Indian_2019',
 'Total_NH_Asian_2019',
 'Total_NH_Pacific_Islander_2019',
 'Total_NH_Two_or_More_Races_2019',
 'Total_Hispanic_White_2019',
 'Total_Hispanic_Black_2019',
 'Total_Hispanic_Two_or_More_Races_2019',
 'CDBG19',
 'HOME19',
 'ESG19',
 'HOPWA19',
 'HTF19',
 'CDBG20',
 'RHP20',
 'HOME20',
 'ESG20',
 'HOPWA20',
 'HTF20',
 'CDBG_CV1',
 'CDBG_CV2',
 'CDBG_CV3',
 'ESG_CV1',
 'ESG_CV2',
 'HOPWA_CV']]

In [470]:
### load in treasury ERAP data 
state_names = pd.read_csv("https://raw.githubusercontent.com/jasonong/List-of-US-States/master/states.csv")

erap1 = pd.read_excel("raw_data/November-ERA-Data.xlsx",sheet_name="ERA1 State & Local Reporting")
erap2 = pd.read_excel("raw_data/November-ERA-Data.xlsx",sheet_name="ERA2 State & Local Reporting")

In [471]:
erap1 = make_header(erap1)
erap2 = make_header(erap2)
erap1["State"] = ""
erap2["State"] = ""

excel_file_erap1 = 'raw_data/November-ERA-Data.xlsx' 
wb_erap1 = load_workbook(excel_file_erap1, data_only = True)
sh_erap1 = wb_erap1['ERA1 State & Local Reporting']

#need to add labels for states - here im looking at color changes as in 
#excel sheet each state is highlighted, so we can change label we want whenever, we index
#through and see a new color 
state_name = "California"
for i in range(erap1.shape[0]):
    if i == 0:
        continue
    color_in_hex =sh_erap1['A'+str(i)].fill.start_color.tint
    if color_in_hex > 0:
        state_name = sh_erap1['A'+str(i)].value
    erap1.loc[i, "State"] = state_name    
    
excel_file_erap2 = 'raw_data/November-ERA-Data.xlsx' 
wb_erap2 = load_workbook(excel_file_erap2, data_only = True)
sh_erap2 = wb_erap1['ERA2 State & Local Reporting']

state_name = "California"
for i in range(erap2.shape[0]):
    if i == 0:
        continue
    color_in_hex =sh_erap2['A'+str(i)].fill.start_color.tint
    if color_in_hex > 0:
        state_name = sh_erap2['A'+str(i)].value
    erap2.loc[i,"State"] = state_name   
erap1 = make_header(erap1)
erap2 = make_header(erap2)
erap1.rename(columns={"California":"State"},inplace=True)
erap2.rename(columns={"California":"State"},inplace=True)
erap1 = erap1[erap1.Location != "Grand Total"]
erap2 = erap2[erap2.Location != "Grand Total"]
erap1=erap1.dropna(axis=1, how='all')
erap2=erap2.dropna(axis=1, how='all')
#merge data and drop DC 
erap1 = erap1.merge(state_names,on="State")
erap2 = erap2.merge(state_names,on="State")
erap1 = erap1[erap1.Location != "State/DC"]
erap2 = erap2[erap2.Location != "State/DC"]

#sum up funds and number of houesholds funds disbursed to 
erap1["Total Number of Households ERAP1 (Direct Allocation)"] = erap1[[x for x in erap1 if "Total" in x]].sum(axis=1)
erap2["Total Number of Households ERAP2 (Direct Allocation)"] = erap2[[x for x in erap2 if "Total" in x]].sum(axis=1)
erap1["ERAP1_total_funds (Direct Allocation)"] = erap1[[x for x in erap1.columns if "Amount" in x]].sum(axis=1)
erap2["ERAP2_total_funds (Direct Allocation)"] = erap2[[x for x in erap2.columns if "Amount" in x]].sum(axis=1)
#flag footnote six and then strip out 
#6Local government recipient that has redirected or will redirect ERA 1 funds to their county or state government. only exits in erap1
erap1["ERAP1_redirected_funds"] = np.where(erap1.Location.str.contains("6"),1,0)
erap1.Location = erap1.Location.str.strip("6")

erap1.rename(columns={"Assistance to Households Expenditure Ratio as of November 305,7":"Assistance to Households Expenditure Ratio as of November ERAP_1 (Direct Allocation)"},inplace=True)
#need to calculate this for ERAP 2 because no longer provided- calcuate figure
#for ERAP2 using footnote 5 for ERAP 1 "5Calculated as the sum of Q1, April, May, June, July, August, September, October, and November expenditure for assistance to households divided by 90% of the ERA1 allocation amount."
erap2["Assistance to Households Expenditure Ratio as of November ERAP_2 (Direct Allocation)"] = erap2["ERAP2_total_funds (Direct Allocation)"]/ (erap2["Full ERA2 Allocation, Award, and Disbursement \n$ Millions4"] * 0.9)

total = erap1.merge(erap2,on=["Location","State","Abbreviation"],how="outer")

#keep only cols i want to merge onto the other 990 data
total_subset = total[["Location","ERAP1_total_funds (Direct Allocation)",'Assistance to Households Expenditure Ratio as of November ERAP_1 (Direct Allocation)',
      'Total Number of Households ERAP1 (Direct Allocation)','Assistance to Households Expenditure Ratio as of November ERAP_2 (Direct Allocation)',
      'ERAP2_total_funds (Direct Allocation)','Total Number of Households ERAP2 (Direct Allocation)',"ERAP1_redirected_funds","State","Abbreviation"]]

total_subset_merge = total_subset.rename(columns={"Location":"County"})

total_subset_merge["County Name"] = total_subset_merge["County"] +" "+ total_subset_merge["Abbreviation"]
total_subset_merge = total_subset_merge.drop(columns={"County","State","Abbreviation"})
#fix mis name 
total_subset_merge.replace({"County Name":{
                                         'Dekalb County GA':"DeKalb County GA"}},inplace=True)

#clean up and fix type of expenditure ratio 
total_subset_merge['Assistance to Households Expenditure Ratio as of November ERAP_1 (Direct Allocation)'] = total_subset_merge['Assistance to Households Expenditure Ratio as of November ERAP_1 (Direct Allocation)'].astype(str).str.strip("%")
total_subset_merge['Assistance to Households Expenditure Ratio as of November ERAP_2 (Direct Allocation)'] = total_subset_merge['Assistance to Households Expenditure Ratio as of November ERAP_2 (Direct Allocation)'].astype(str).str.strip("%")
total_subset_merge['Assistance to Households Expenditure Ratio as of November ERAP_1 (Direct Allocation)'] = total_subset_merge['Assistance to Households Expenditure Ratio as of November ERAP_1 (Direct Allocation)'].astype(str).str.strip("+")
total_subset_merge['Assistance to Households Expenditure Ratio as of November ERAP_2 (Direct Allocation)'] = total_subset_merge['Assistance to Households Expenditure Ratio as of November ERAP_2 (Direct Allocation)'].astype(str).str.strip("+")
total_subset_merge['Assistance to Households Expenditure Ratio as of November ERAP_1 (Direct Allocation)'] = total_subset_merge['Assistance to Households Expenditure Ratio as of November ERAP_1 (Direct Allocation)'].astype(float)
total_subset_merge['Assistance to Households Expenditure Ratio as of November ERAP_2 (Direct Allocation)'] = total_subset_merge['Assistance to Households Expenditure Ratio as of November ERAP_2 (Direct Allocation)'].astype(float)

### now add in scraped state allocation county data 

In [472]:
#read in state allocation data (from various state websites)
#text is scraped in various ways, housed in this notebook ERAP_Extraction_Main_1_17_22
final_erap_simple = pd.read_csv("processed_data/1_26_22_ERAP_county_main.csv")

final_erap_simple.County = final_erap_simple.County.str.title()

def fix_county(x):
    if "County" in x:
        return x
    if "Parish" in x:
        return x
    if "Borough" in x:
        return x
    else:
        return x + " County"
    
final_erap_simple["County"] = final_erap_simple["County"].apply(lambda x: fix_county(x))

final_erap_simple.rename(columns={"Number of Payments":"Number of Payments (State Allocation)",
                                 "Total Amount of Payments":"Total Amount of Payments (State Allocation)",
                                 "Average Payment Amount":"Average Payment Amount (State Allocation)"},inplace=True)

# add on state abriviations 
final_erap_simple = final_erap_simple.merge(state_names,on="State",how="left")
final_erap_simple["County Name"] = final_erap_simple["County"] +" "+ final_erap_simple["Abbreviation"]
final_erap_simple = final_erap_simple.drop(columns={"Unnamed: 0","County","State","Abbreviation"})

#fix names 
final_erap_simple.replace({"County Name":{'Dearbon County IN':'Dearborn County IN',
                                         'St. Louis City County MO':"St. Louis city MO",
                                         'Lagrange County IN':"LaGrange County IN",
                                         'Dekalb County IN':"DeKalb County IN",
                                         'Dekalb County MO':'DeKalb County MO',
                                         'Laporte County IN':"LaPorte County IN",
                                         'Mcdonald County MO':"McDonald County MO",
                                         'Mcpherson County KS':"McPherson County KS",
                                         'St. John The Baptist Parish LA':"St. John the Baptist Parish LA",
                                          'Lake Of The Woods County MN':"Lake of the Woods County MN",
                                          "Mcleod County MN":"McLeod County MN",
                                          "Lac Qui Parle County MN":"Lac qui Parle County MN",
                                         "Atascosa  County TX":"Atascosa County TX",
                                         'Dewitt County TX':"DeWitt County TX",
                                         'Kelberg County TX':"Kleberg County TX",
                                         'Mcculloch County TX':'McCulloch County TX',
                                         'Mclennan County TX':"McLennan County TX",
                                         'Mcmullen County TX':"McMullen County TX",
                                         'Anchorage Municipality  County AK':"Anchorage Municipality AK",
 'Anchorage Municipality County AK':"Anchorage Municipality AK",
 'Baltimore City County MD':"Baltimore city MD",
 'Juneau City And Borough AK':'Juneau City and Borough AK',
 'Lake And Peninsula Borough AK':"Lake and Peninsula Borough AK",
 'Matanuska-Sustna Borough AK':"Matanuska-Susitna Borough AK",
 'Pisctaquis County ME':"Piscataquis County ME",
 "Prince George'S County MD":"Prince George's County MD",
 "Queen Anne'S County MD":"Queen Anne's County MD",
 'Sagauche County CO':"Saguache County CO",
 "Saint Mary'S County MD":"St. Mary's County MD",
 'Skagway Municipality County AK':"Skagway Municipality AK",
 'Wexfor County MI':"Wexford County MI",
 'Wrangell City And Borough AK':"Wrangell City and Borough AK",
 'Yakutat City And Borough AK':"Yakutat City and Borough AK"}},inplace=True)

#full_990 = pd.read_csv("/Users/underriner/Desktop/work/990_scraping/1_9_22_990_data_county_level_full_US.csv")
#merge on state abrivations 
#full_990 = full_990.merge(state_names,on="State")

In [473]:
### note maybe switch RI from counties to places 

In [474]:
#merge both tresuary and state datasets onto 990 data 
data_990_final_merged = data_990_final.merge(final_erap_simple,on="County Name",how="left")
data_990_final_merged = data_990_final_merged.merge(total_subset_merge,on="County Name",how="left")
data_990_final_merged = data_990_final_merged.drop_duplicates(subset=["County Name"])#DeKalb County GA was fully duplicated, not sure why

In [475]:
#data_990_final_merged.to_csv("1_17_22_county_interim_save.csv")

In [476]:
#fix hispanic figure to be all hispanic 
hispanic = pd.read_csv("raw_data/hispanic_county.csv")#from social explorer (SE B04001)
hispanic["Geo_STUSAB"] = hispanic["Geo_STUSAB"].str.upper()
hispanic["cntyname"] = hispanic["Geo_NAME"]+" " + hispanic["Geo_STUSAB"]
hispanic = hispanic[["cntyname","SE_B04001_010"]]
hispanic.rename(columns={"SE_B04001_010":"cnty_hispanic",
                        "cntyname":"County Name"},inplace=True)
hispanic.replace({"County Name":{"LaSalle Parish LA":"La Salle Parish LA"}},inplace=True)
data_990_final_merged = data_990_final_merged.merge(hispanic,on="County Name",how="left")

In [477]:
data_990_final_merged.rename(columns={'County Name':"cntyname",
                      "Total_pop_2019":"cnty_pop",
                      "Total_NH_White_2019":"cnty_white",
                      "Total_NH_Black_2019":"cnty_black",
                      "Total_NH_American_Indian_2019":"cnty_amin",
                      "Total_NH_Asian_2019":"cnty_asian",
                      "Total_NH_Pacific_Islander_2019":"cnty_nhpil",
                      "Total_NH_Two_or_More_Races_2019":"cnty_other",
                      "Total_pop_poverty_universe_2019":"cnty_povpop",
                      "Total_pop_below_poverty_line_2019":"cnty_belowpov",
                      "Total_Households_2019":"cnty_households",
                      "Total_Owner_Occupied_2019":"cnty_owners",
                      "Total_Renter_Occupied_2019":"cnty_renters",
                      "Percent_Below_Poverty_Line":"cnty_pov",
                      "Number of Nonprofits":"nonprofits",
                      "Total Assets of Nonprofits":"np_assets",
                      "Total Income of Nonprofits in Area":"np_income",
                      "Assets_per_capita":"np_assets_pc",
                      "Income_per_capita":"np_income_pc",
                      "Nonprofits_per_1k":"nonprofits_1k",
                     "Percent_Sub_50%_AMI":'cnty_sub50_ami',
                    "Percent_Sub_50%_AMI_Renter":'cnty_sub50_ami_renter',
                      "CDBG20":"cnty_CDBG20",
                      "HOME20":"cnty_HOME20",
                      "ESG20":"cnty_ESG20",
                      "CDBG_CV1":"cnty_CDBG_CV1",
                      "CDBG_CV2":"cnty_CDBG_CV2",
                      "CDBG_CV3":"cnty_CDBG_CV3",
                      "ESG_CV1":"cnty_ESG_CV1",
                      "ESG_CV2":"cnty_ESG_CV2",
                      "CDBG19":"cnty_CDBG19",
                      "HOME19":"cnty_HOME19",
                      "ESG19":"cnty_ESG19",
                      "HOPWA19":"cnty_HOPWA19",
                      "HTF19":"cnty_HTF19",
                        "RHP20":"cnty_RHP20",
                        'HOPWA20':"cnty_HOPWA20", 
                        'HTF20':'cnty_HTF20',
                            "HOPWA_CV":"cnty_HOPWA_CV",
                      "Number of Payments (State Allocation)":"cnty_erap_payment_count_state_allocation",
                      "Total Amount of Payments (State Allocation)":"cnty_erap_total_amt_state_allocation",
                      "Average Payment Amount (State Allocation)":"cnty_erap_avg_amt_state_allocation",
                      "ERAP1_total_funds (Direct Allocation)":"cnty_erap1_funds",
                      "Assistance to Households Expenditure Ratio as of November ERAP_1 (Direct Allocation)":"cnty_ratio_erap_1",
                      "Total Number of Households ERAP1 (Direct Allocation)":"cnty_hh_erap_1",
                      "Total Number of Households ERAP2 (Direct Allocation)":"cnty_hh_erap_2",
                      "Assistance to Households Expenditure Ratio as of November ERAP_2 (Direct Allocation)":"cnty_ratio_erap_2",
                      "ERAP2_total_funds (Direct Allocation)":"cnty_erap2_funds",
                                     "ERAP1_redirected_funds":"cnty_ERAP1_redirected_funds"},inplace=True)

In [478]:
data_990_final_merged["cnty_pov"] = data_990_final_merged["cnty_belowpov"]/data_990_final_merged["cnty_povpop"]
data_990_final_merged["stab"] = data_990_final_merged.cntyname.str[-2:]

In [479]:
# add in children demographic data 
children = pd.read_csv("raw_data/county_children_mapping.csv",dtype={"Geo_FIPS":str}) #from social explorer, B25012, B25115
children.rename(columns={"Geo_FIPS":"FIPS","ACS19_5yr_B25012011":"cnty_renter_child_households",
                     "ACS19_5yr_B25115021":"cnty_renter_child_single_parent_households_male",
                     "ACS19_5yr_B25115024":"cnty_renter_child_single_parent_households_female"},inplace=True)
children["cnty_renter_child_single_parent_households"] = children["cnty_renter_child_single_parent_households_male"] + children["cnty_renter_child_single_parent_households_female"] 
children = children[["FIPS","cnty_renter_child_households","cnty_renter_child_single_parent_households"]]
data_990_final_merged = data_990_final_merged.merge(children,on="FIPS",how="left")

In [480]:
data_990_final_merged = data_990_final_merged[['cntyname','stab', 'FIPS', 'nonprofits', 'np_assets', 'np_income',
       'np_assets_pc', 'np_income_pc', 'nonprofits_1k', 'cnty_med_income',
       'cnty_med_rent', 'p_unemployed_Feb20', 'p_unemployed_Feb21',
       'p_unemployed_Oct21', 'p_unemployed_Nov21', 'cnty_povpop',
       'cnty_belowpov', 'cnty_pov','cnty_pop', 'cnty_households', 'cnty_owners',
       'cnty_renters', "cnty_renter_child_households","cnty_renter_child_single_parent_households", 
        'cnty_white', 'cnty_black', 'cnty_hispanic',
        'cnty_amin', 'cnty_asian',
       'cnty_nhpil', 'cnty_other','cnty_sub50_ami', 'cnty_sub50_ami_renter', 
       'cnty_CDBG19', 'cnty_HOME19', 'cnty_ESG19',
       'cnty_HOPWA19', 'cnty_HTF19', 'cnty_CDBG20', 'cnty_RHP20',
       'cnty_HOME20', 'cnty_ESG20', 'cnty_HOPWA20', 'cnty_HTF20',
       'cnty_CDBG_CV1', 'cnty_CDBG_CV2', 'cnty_CDBG_CV3', 'cnty_ESG_CV1',
       'cnty_ESG_CV2', 'cnty_HOPWA_CV', 'cnty_erap_payment_count_state_allocation',
       'cnty_erap_total_amt_state_allocation', 'cnty_erap_avg_amt_state_allocation',
       'cnty_erap1_funds', 'cnty_ratio_erap_1', 'cnty_hh_erap_1',
       'cnty_ratio_erap_2', 'cnty_erap2_funds', 'cnty_hh_erap_2',
       'cnty_ERAP1_redirected_funds']]

In [481]:
#final county  
#data_990_final_merged.to_csv("processed_data/cnty_ERAP_1_26_22.csv")

In [482]:
data_990_final_merged[data_990_final_merged["p_unemployed_Nov21"].isna()]

,cntyname,stab,FIPS,nonprofits,np_assets,np_income,np_assets_pc,np_income_pc,nonprofits_1k,cnty_med_income,cnty_med_rent,p_unemployed_Feb20,p_unemployed_Feb21,p_unemployed_Oct21,p_unemployed_Nov21,cnty_povpop,cnty_belowpov,cnty_pov,cnty_pop,cnty_households,cnty_owners,cnty_renters,cnty_renter_child_households,cnty_renter_child_single_parent_households,cnty_white,cnty_black,cnty_hispanic,cnty_amin,cnty_asian,cnty_nhpil,cnty_other,cnty_sub50_ami,cnty_sub50_ami_renter,cnty_CDBG19,cnty_HOME19,cnty_ESG19,cnty_HOPWA19,cnty_HTF19,cnty_CDBG20,cnty_RHP20,cnty_HOME20,cnty_ESG20,cnty_HOPWA20,cnty_HTF20,cnty_CDBG_CV1,cnty_CDBG_CV2,cnty_CDBG_CV3,cnty_ESG_CV1,cnty_ESG_CV2,cnty_HOPWA_CV,cnty_erap_payment_count_state_allocation,cnty_erap_total_amt_state_allocation,cnty_erap_avg_amt_state_allocation,cnty_erap1_funds,cnty_ratio_erap_1,cnty_hh_erap_1,cnty_ratio_erap_2,cnty_erap2_funds,cnty_hh_erap_2,cnty_ERAP1_redirected_funds
548,Kalawao County HI,HI,15005,NaN,NaN,NaN,NaN,NaN,NaN,69375,1063.00,NaN,NaN,NaN,NaN,60,6,0.10,66,39,0,39,0,0,15,2,2,1,14,18,14,18.22,36.03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [483]:
hispanic.replace({"County Name":{"LaSalle Parish LA":"La Salle Parish LA"}},inplace=True)

In [484]:
hispanic[hispanic["County Name"].str.contains("LaS")]

,County Name,cnty_hispanic
644,LaSalle County IL,10658


In [485]:
#so med rent data is missing from ACS for those 4 places 
# BLS data is missing for Kalawao County HI
addme[addme.FIPS.isin(['32011', '48033', '48301', '48443'])]

,FIPS,cnty_med_income,cnty_med_rent
1752,32011,67105,NaN
2539,48033,72188,NaN
2673,48301,83750,NaN
2744,48443,43208,NaN


In [496]:
data_990_final_merged[data_990_final_merged["cnty_sub50_ami"].isna()]

,cntyname,stab,FIPS,nonprofits,np_assets,np_income,np_assets_pc,np_income_pc,nonprofits_1k,cnty_med_income,cnty_med_rent,p_unemployed_Feb20,p_unemployed_Feb21,p_unemployed_Oct21,p_unemployed_Nov21,cnty_povpop,cnty_belowpov,cnty_pov,cnty_pop,cnty_households,cnty_owners,cnty_renters,cnty_renter_child_households,cnty_renter_child_single_parent_households,cnty_white,cnty_black,cnty_hispanic,cnty_amin,cnty_asian,cnty_nhpil,cnty_other,cnty_sub50_ami,cnty_sub50_ami_renter,cnty_CDBG19,cnty_HOME19,cnty_ESG19,cnty_HOPWA19,cnty_HTF19,cnty_CDBG20,cnty_RHP20,cnty_HOME20,cnty_ESG20,cnty_HOPWA20,cnty_HTF20,cnty_CDBG_CV1,cnty_CDBG_CV2,cnty_CDBG_CV3,cnty_ESG_CV1,cnty_ESG_CV2,cnty_HOPWA_CV,cnty_erap_payment_count_state_allocation,cnty_erap_total_amt_state_allocation,cnty_erap_avg_amt_state_allocation,cnty_erap1_funds,cnty_ratio_erap_1,cnty_hh_erap_1,cnty_ratio_erap_2,cnty_erap2_funds,cnty_hh_erap_2,cnty_ERAP1_redirected_funds
1163,St. Mary Parish LA,LA,22101,11.00,10780924.00,11392031.00,218.21,230.58,0.22,40485,781.00,5.20,8.80,5.20,5.10,50119,9890,0.20,50968,19856,12056,7800,1799,1543,28581,15432,3585,787,734,11,1796,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.00,163934.00,4683.83,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [497]:
data_990_final_merged.isna().sum()

cntyname                                         0
stab                                             0
FIPS                                             0
nonprofits                                     368
np_assets                                      368
np_income                                      368
np_assets_pc                                   369
np_income_pc                                   369
nonprofits_1k                                  369
cnty_med_income                                  0
cnty_med_rent                                    4
p_unemployed_Feb20                               1
p_unemployed_Feb21                               1
p_unemployed_Oct21                               1
p_unemployed_Nov21                               1
cnty_povpop                                      0
cnty_belowpov                                    0
cnty_pov                                         0
cnty_pop                                         0
cnty_households                

In [488]:
data_990_final_merged[data_990_final_merged["cnty_med_rent"].isna()].FIPS.unique()

array(['32011', '48033', '48301', '48443'], dtype=object)

In [489]:
HUD_INCOME_LIMITS_2019[HUD_INCOME_LIMITS_2019.county_name.str.contains("Mary")]["county_name"][1324]

'St. Mary Parish'

In [490]:
data_990_final_merged[data_990_final_merged["cnty_sub50_ami"].isna()]["cntyname"][1163]

'St. Mary Parish LA'

In [491]:
#need to rename ERAP columns/ find out why that didnt happen 
data_990_final_merged.isna().sum()

cntyname                                         0
stab                                             0
FIPS                                             0
nonprofits                                     368
np_assets                                      368
np_income                                      368
np_assets_pc                                   369
np_income_pc                                   369
nonprofits_1k                                  369
cnty_med_income                                  0
cnty_med_rent                                    4
p_unemployed_Feb20                               1
p_unemployed_Feb21                               1
p_unemployed_Oct21                               1
p_unemployed_Nov21                               1
cnty_povpop                                      0
cnty_belowpov                                    0
cnty_pov                                         0
cnty_pop                                         0
cnty_households                

In [499]:
data_990_final_merged.to_csv("processed_data/cnty_ERAP_1_26_22")

In [492]:
#data_990_final.to_csv("11_4_21_990_data_county_level_full_US.csv")

In [493]:
#data_990_final.to_csv("1_9_22_990_data_county_level_full_US.csv")

In [494]:
#data_990_final.drop(columns=["Total_pop_2020"],inplace=True)

In [495]:
#now fold through to ERAP_Allocation_addition_11_29_21 